# Chapter 2
Chapter 2 focuses on the introduction of between-person analysis. The data used in this example are a subset of the [Octogenarian Twin Study of Aging](https://www.maelstrom-research.org/study/octo-twin) dataset. They consist of data from 550 older adults, for which the following measurements were collected:
- age
- cognition (assessed with the Information Test)
- grip strength
- sex
- dementia diagnosis

A review of general linear models (GLM) and Analysis of Variance (ANOVA) methods is provided in this chapter, with a particular focus on the interpretation of interactions among continuous and categorical predictors.

## Import packages

In [50]:
import math
import os

import pandas as pd
from sas7bdat import SAS7BDAT
import scipy.stats
import statsmodels.formula.api
import statsmodels.stats.anova

## Constants

In [2]:
# File paths
FILE_PATH = os.path.join("Data", "SAS_Chapter2.sas7bdat")

# File columns
AGE_COL = "age"
COGNITION_COL = "cognition"
GRIP_COL = "grip"

## Read data

In [3]:
with SAS7BDAT(FILE_PATH, skip_header=False) as reader:
    df = reader.to_data_frame()

# Between-Person Empty Model
Let's now start our review of between-person analysis methods using an empty model, that does not use any predictor to predict cognition:

$
y_i = \beta_0 + e_i
$

The only parameter that our model needs to determine is the intercept $\beta_0$.

In [4]:
empty_model = statsmodels.formula.api.ols(formula="cognition ~ 1", data=df).fit()
empty_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cognition   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 18 Jul 2024   Prob (F-statistic):                nan
Time:                        16:18:51   Log-Likelihood:                -2098.2
No. Observations:                 550   AIC:                             4198.
Df Residuals:                     549   BIC:                             4203.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.8218      0.469     52.973      0.000      23.901      25.742
==============================================================================
Omnibus:                       23.202   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               14.625
Skew:                          -0.260   Prob(JB):                     0.000667
Kurtosis:                       2.393   Cond. No.                         1.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

As we can see from the results above, the intercept (`24.82`) corresponds to the grand total mean of cognition across all the dataset:

In [12]:
print(f"Grand mean for cognition: {df[COGNITION_COL].mean():.2f}")
print(f"Intercept of the empty model: {empty_model.params.loc["Intercept"]:.2f}")

Grand mean for cognition: 24.82
Intercept of the empty model: 24.82


The variance of the residuals is computed as:

$
\sigma_e^2 = \frac{\sum_{i=1}^N\left(y_i-\hat{y_i}^2\right)}{N-k}
$

where $N$ is the total number of samples, $k$ is the number of effects (1 in this case, since we only have the intercept).

With `statsmodels`, we can retrieve the residual variance by accessing ``empty_model.mse_resid``, and for the empty model it corresponds to 120.76.

In [14]:
print(f"Residual variance for the empty model: {empty_model.mse_resid:.2f}")

Residual variance for the empty model: 120.76


# Between-Person Analysis Using Continuous Predictors

## Age
We expect age to be a good predictor of cognition, with a decrease in cognition as age progresses. We can then build a model that uses age to predict the cognition. 

Since the samples of our dataset contain data from people with age > 80, we need to center the age predictor in order to have a meaningful 0 value: we will use 85 as a 0 value.
We substract this value to create a new variable named `centered_age`, and we will use it in our models.

In [15]:
CENTERED_AGE_COL = "centered_age"
df[CENTERED_AGE_COL] = df[AGE_COL] - 85

Let's now fit the model using centered_age as a predictor.

In [17]:
age_model = statsmodels.formula.api.ols(formula=f"{COGNITION_COL} ~ {CENTERED_AGE_COL}", data=df).fit()
age_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cognition   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     16.40
Date:                Thu, 18 Jul 2024   Prob (F-statistic):           5.87e-05
Time:                        16:25:03   Log-Likelihood:                -2090.1
No. Observations:                 550   AIC:                             4184.
Df Residuals:                     548   BIC:                             4193.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       24.7818      0.462     53.612      0.000      23.874      25.690
centered_age    -0.5461      0.135     -4.049      0.000      -0.811      -0.281
==============================================================================
Omnibus:                       19.700   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.788
Skew:                          -0.238   Prob(JB):                      0.00167
Kurtosis:                       2.425   Cond. No.                         3.43
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The coefficient for centered_age is equal to -0.5461, and as expected it is a negative coefficient, meaning that there is a decline in cognition as age progresses. In particular, for a one-unit change in age, there is an expected decrease in cognition of -0.5461.

The intercept value (now equal to 24.7818) corresponds to the expected cognition value for a person with 85 years old, since our 0 is now 85 years

The $R^2$ of the model that uses age as a unique predictor is 0.03. Another way to get to the same value it's by computing the relative change in residual variance with respect to the empty model:

```
(age_model.mse_resid - empty_model.mse_resid)/empty_model.mse_resid
```


In [18]:
print(f"{(empty_model.mse_resid-age_model.mse_resid)/empty_model.mse_resid:.2f}")

0.03


In order to determine if the age coefficient is significat, we need to compute the standard error for the coefficient. The standard error for a coefficient is defined as:

$
SE_{\beta_X} = \sqrt{\frac{Var(y_i)*(1-R_Y^2)}{Var(x_i)*(1-R_x^2)*(N-k)}}
$

Thus, the standard error ($SE$) for a predictor depends on the amount of variance in the outcome variable that is still remaining in the model

In [35]:
# SE numerator
se_num = df[COGNITION_COL].std()**2*(1-((empty_model.mse_resid-age_model.mse_resid)/empty_model.mse_resid))
# SE denominator
se_den = (df[AGE_COL].std()**2)*(1-0)*(len(df)-1)
# Compute SE
se = math.sqrt(se_num/se_den)
print(f"SE: {se:.3f}")

SE: 0.135


In order to determine if the slope is significantly different from $0$, we can run what it's called a Wald test. We compute the ratio between the coefficient and its standard error (SE), and compare it a t-distribution to determine if the age slope is different from 0.

In [55]:
t_statistic = age_model.params.loc[CENTERED_AGE_COL]/se
print(f"t-statistic: {t_statistic:.4f}")

t-statistic: -4.0494


In [57]:
critical_value = scipy.stats.t.ppf(0.025,len(df)-2)
print(t_statistic < critical_value)

True


We can also determine the confidence interval for the estimate of the coefficient, defined as $\beta_1\pm1.96*SE$:

In [65]:
print(f"Age coefficient CI: ({age_model.params.loc[CENTERED_AGE_COL]-1.96*se:.4f},{age_model.params.loc[CENTERED_AGE_COL]+1.96*se:.4f})")

Age coefficient CI: (-0.8104,-0.2818)


## Age and Grip
In addition to age, we can include another predictor in the model that is the grip strength. Similar to what we did for age, we need to properly center the grip strength. Since we have a mean of around 9 pounds, we use this value as a center point.

In [37]:
CENTERED_GRIP_COL = "centered_grip"
df[CENTERED_GRIP_COL] = df[GRIP_COL] - 9

In [66]:
age_grip_model = statsmodels.formula.api.ols(formula=f"""{COGNITION_COL} ~ 
                                             {CENTERED_AGE_COL} + 
                                             {CENTERED_GRIP_COL}""", data=df).fit()
age_grip_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cognition   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     22.14
Date:                Thu, 18 Jul 2024   Prob (F-statistic):           5.66e-10
Time:                        17:11:27   Log-Likelihood:                -2076.8
No. Observations:                 550   AIC:                             4160.
Df Residuals:                     547   BIC:                             4173.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        24.7008      0.452     54.662      0.000      23.813      25.588
centered_age     -0.4176      0.134     -3.115      0.002      -0.681      -0.154
centered_grip     0.8025      0.154      5.206      0.000       0.500       1.105
==============================================================================
Omnibus:                       17.496   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.268
Skew:                          -0.248   Prob(JB):                      0.00217
Kurtosis:                       2.462   Cond. No.                         3.57
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now, the intercept is the expected cognition level for a person of 85 years old with 9 pounds grip strength. The slope for age ($-0.4176$) is less than the previous one, since we have a correlation with grip strength in the dataset.

With the grip strength coefficient of 0.80, we expect cognition to increase of 0.8 for every additional year of age.

In [67]:
df.loc[:,[CENTERED_AGE_COL, CENTERED_GRIP_COL]].corr()

,centered_age,centered_grip
centered_age,1.000000,-0.184135
centered_grip,-0.184135,1.000000


In [34]:
print(f"{(empty_model.mse_resid-age_grip_model.mse_resid)/empty_model.mse_resid:.2f}")

0.07


# Between-Person Analysis Using Categorical Predictors

In [45]:
df["demgroup"].value_counts()

demgroup
1.0    399
2.0    109
3.0     42
Name: count, dtype: int64

In [79]:
## 
sex_categorical = pd.CategoricalDtype(categories=[1,0],ordered=True)
dementia_categorical = pd.CategoricalDtype(categories=[3,1,2],ordered=True)

df["cat_sex"] = df["sexMW"].astype(sex_categorical)
df["cat_dementia"] = df["demgroup"].astype(dementia_categorical)
# 1 : 1, 2 : 2, 3 : 0 NO
# 1 : 0, 2 : 2, 3 : 1 NO
# 1 : 2, 2 : 1, 3 : 0 NO
#df["cat_dementia"] = df["demgroup"].map({1:2,2:1,3:0})
age_grip_model = statsmodels.formula.api.ols(formula="cognition ~ centered_age + centered_grip + centered_age*centered_grip + cat_sex + cat_dementia + centered_age*centered_grip + cat_sex*cat_dementia", data=df).fit()
age_grip_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cognition   R-squared:                       0.298
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                     28.77
Date:                Mon, 15 Jul 2024   Prob (F-statistic):           2.11e-37
Time:                        23:54:42   Log-Likelihood:                -2000.7
No. Observations:                 550   AIC:                             4019.
Df Residuals:                     541   BIC:                             4058.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          6.3487      1.948      3.259      0.001       2.522      10.175
cat_sex[T.0]                      10.7507      2.899      3.708      0.000       5.055      16.446
cat_dementia[T.1]                 19.8458      2.029      9.783      0.000      15.861      23.831
cat_dementia[T.2]                 13.9542      2.239      6.233      0.000       9.556      18.352
cat_sex[T.0]:cat_dementia[T.1]    -7.8751      3.025     -2.604      0.009     -13.816      -1.934
cat_sex[T.0]:cat_dementia[T.2]    -8.0394      3.415     -2.354      0.019     -14.748      -1.331
centered_age                      -0.3348      0.120     -2.793      0.005      -0.570      -0.099
centered_grip                      0.6179      0.148      4.173      0.000       0.327       0.909
centered_age:centered_grip         0.1222      0.040      3.027      0.003       0.043       0.201
==============================================================================
Omnibus:                        9.364   Durbin-Watson:                   1.954
Prob(Omnibus):                  0.009   Jarque-Bera (JB):                8.073
Skew:                          -0.226   Prob(JB):                       0.0177
Kurtosis:                       2.616   Cond. No.                         149.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [69]:
statsmodels.stats.anova.anova_lm(age_grip_model)

,df,sum_sq,mean_sq,F,PR(>F)
C(cat_sex),1.0,3701.363784,3701.363784,43.052923,1.248467e-10
C(cat_dementia),2.0,11683.820315,5841.910157,67.950983,4.705848e-27
C(cat_sex):C(cat_dementia),2.0,621.962400,310.981200,3.617221,2.750881e-02
centered_age,1.0,1564.232502,1564.232502,18.194586,2.354246e-05
centered_grip,1.0,1426.295716,1426.295716,16.590155,5.333268e-05
centered_age:centered_grip,1.0,787.786754,787.786754,9.163250,2.586842e-03
Residual,541.0,46511.076711,85.972415,NaN,NaN


In [80]:
age_grip_model.wald_test_terms(scalar=True)

<class 'statsmodels.stats.contrast.WaldTestResults'>
                                    F           P>F  df constraint  df denom
Intercept                   10.623021  1.187067e-03              1     541.0
cat_sex                     13.749273  2.304491e-04              1     541.0
cat_dementia                53.157317  8.377946e-22              2     541.0
cat_sex:cat_dementia         3.491936  3.112981e-02              2     541.0
centered_age                 7.798652  5.413560e-03              1     541.0
centered_grip               17.411527  3.506457e-05              1     541.0
centered_age:centered_grip   9.163250  2.586842e-03              1     541.0